In [1]:
# !pip install ultralytics

In [9]:
import cv2
from ultralytics import YOLO
import numpy as np
import torch


In [2]:
model = YOLO("best_aug.pt")

In [10]:
from numpy.lib.npyio import savetxt
from torch.serialization import save
predict = model.predict("/home/avinash/Projects/video_masking/image/77.png", save = True , save_txt = True)


image 1/1 /home/avinash/Projects/video_masking/image/77.png: 384x640 1 window, 17.2ms
Speed: 11.0ms preprocess, 17.2ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/segment/predict30
1 label saved to runs/segment/predict30/labels


In [11]:
type(predict[0])

ultralytics.engine.results.Results

In [12]:
(predict[0].masks.data[0].cpu().numpy() *255).astype("uint8")

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [13]:
cv2.imshow("mask",(predict[0].masks.data[0].cpu().numpy() *255).astype("uint8"))

QObject::moveToThread: Current thread (0x95a1a0a0) is not the object's thread (0x95b78350).
Cannot move to target thread (0x95a1a0a0)

QObject::moveToThread: Current thread (0x95a1a0a0) is not the object's thread (0x95b78350).
Cannot move to target thread (0x95a1a0a0)

QObject::moveToThread: Current thread (0x95a1a0a0) is not the object's thread (0x95b78350).
Cannot move to target thread (0x95a1a0a0)

QObject::moveToThread: Current thread (0x95a1a0a0) is not the object's thread (0x95b78350).
Cannot move to target thread (0x95a1a0a0)

QObject::moveToThread: Current thread (0x95a1a0a0) is not the object's thread (0x95b78350).
Cannot move to target thread (0x95a1a0a0)

QObject::moveToThread: Current thread (0x95a1a0a0) is not the object's thread (0x95b78350).
Cannot move to target thread (0x95a1a0a0)

QObject::moveToThread: Current thread (0x95a1a0a0) is not the object's thread (0x95b78350).
Cannot move to target thread (0x95a1a0a0)

QObject::moveToThread: Current thread (0x95a1a0a0) is n

In [15]:
cv2.imwrite("mask_77.png",(predict[0].masks.data[0].cpu().numpy() *255).astype("uint8"))

True

In [24]:
data = predict[0].masks.data[0]

In [35]:
img= cv2.imread('Window_Test_Video.mp4')
model = YOLO('best_aug.pt')
results = model.predict(save=True, save_txt=False, stream=True)
for result in results:
    # get array results
    masks = result.masks.masks
    boxes = result.boxes.boxes
    # extract classes
    clss = boxes[:, 5]
    # get indices of results where class is 0 (people in COCO)
    people_indices = torch.where(clss == 0)
    # use these indices to extract the relevant masks
    people_masks = masks[people_indices]
    # scale for visualizing results
    people_mask = torch.any(people_masks, dim=0).int() * 255
    # save to file
    cv2.imwrite(str(model.predictor.save_dir / 'Window_Test_Video.mp4'), people_mask.cpu().numpy())

WARNING ⚠️ 'source' is missing. Using 'source=https://ultralytics.com/images/bus.jpg'.

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
WARNING ⚠️ 'Masks.masks' is deprecated. Use 'Masks.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


error: OpenCV(4.8.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:696: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'


In [15]:
import cv2
import numpy as np

def alpha_mask_video(input_video_path, output_video_path, yolo_model, confidence_threshold=0.5):
    # Open the input video
    input_video = cv2.VideoCapture(input_video_path)
    frame_width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(input_video.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    while True:
        ret, frame = input_video.read()
        if not ret:
            break
        
        # Perform YOLOv8 inference on the frame to get instance segment windows
        # yolo_model.inference(frame) - replace this line with your actual YOLOv8 inference code
        
        # Assuming yolo_output contains the instance segment windows (bounding boxes)
        # yolo_output format: [(class_id, confidence, x, y, width, height), ...]
        
        # Create an alpha mask based on the predicted bounding boxes
        alpha_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        for class_id, confidence, x, y, width, height in yolo_output:
            if confidence > confidence_threshold:
                mask = np.zeros(frame.shape[:2], dtype=np.uint8)
                cv2.rectangle(mask, (x, y), (x + width, y + height), 255, -1)
                alpha_mask = cv2.add(alpha_mask, mask)
        
        # Apply the alpha mask to the frame
        masked_frame = cv2.bitwise_and(frame, frame, mask=alpha_mask)
        
        # Write the masked frame to the output video
        output_video.write(masked_frame)
    
    # Release video objects
    input_video.release()
    output_video.release()
    cv2.destroyAllWindows()

# Replace 'yolo_model.inference(frame)' with your actual YOLOv8 inference code
# You'll need to have your YOLOv8 model and inference logic implemented.

# Example usage
input_video_path = 'Window_Test_Video.mp4'
output_video_path = '/home/avinash/Projects/yolov8/windows_seg'
yolo_model = YOLO('best_aug.pt') # Instantiate and load your YOLOv8 model here
confidence_threshold = 0.5

alpha_mask_video(input_video_path, output_video_path, yolo_model, confidence_threshold)


[ERROR:0@1204.955] global cap.cpp:643 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): /home/avinash/Projects/yolov8/windows_seg in function 'icvExtractPattern'




NameError: name 'yolo_output' is not defined

In [8]:
from ultralytics import YOLO
# from ultralytics.models.yolo import scale_image
import cv2
import numpy as np

In [10]:
# Load a model
model = YOLO("best_aug.pt")
class_names = model.names
print('Class Names: ', class_names)
colors = [[np.random.randint(0, 255) for _ in range(3)] for _ in class_names]
cap = cv2.VideoCapture('Window_Test_Video.mp4')
cap = cv2.VideoCapture(2)

while True:
    success, img = cap.read()
    if not success:
        break

    h, w, _ = img.shape
    results = model.predict(img, stream=True)
    # print(results)
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

    if masks is not None:
        masks = masks.data.cpu()
        for seg, box in zip(masks.data.cpu().numpy(), boxes):

            seg = cv2.resize(seg, (w, h))
            img = overlay(img, seg, colors[int(box.cls)], 0.4)
            
            xmin = int(box.data[0][0])
            ymin = int(box.data[0][1])
            xmax = int(box.data[0][2])
            ymax = int(box.data[0][3])
            
            plot_one_box([xmin, ymin, xmax, ymax], img, colors[int(box.cls)], f'{class_names[int(box.cls)]} {float(box.conf):.3}')

    cv2.imshow('img', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Class Names:  {0: 'window'}


[ WARN:0@3513.413] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@3513.530] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range
